In [ ]:
covid_df = pd.read_csv('/home/gopherguy14/datasets/covid_confirmed_usafacts.csv')
pop_df = pd.read_csv('/home/gopherguy14/datasets/covid_county_population.csv', engine='python')

# create subset of data to show to be one year prior to March 12th, 2020 and
# up to December 31st, 2020 
crashes_subset = crashes_df[ ( crashes_df['Start_Time'] > '2019-03-01' ) & ( crashes_df['Start_Time'] < '2021-01-01') ][['ID','Start_Date','County','State']]
# remove duplicates
crashes_subset = crashes_subset.drop_duplicates(subset='ID')
# perform the aggregation
crashes_subset = crashes_subset.groupby(['Start_Date','County','State']).agg({'ID':'count'})
crashes_subset = crashes_subset.reset_index()

# select the id columns
id_vars = covid_df.columns[:4]
# select only the date columns
value_vars = covid_df.columns[4:]
# melt the covid data
covid_melt_df = pd.melt(covid_df, id_vars=id_vars, value_vars=value_vars)
# find new cases by sorting
covid_melt_df = covid_melt_df.sort_values(by=['countyFIPS','variable'])
# then grouping and subtracting each successive date from the next
covid_melt_df['new_cases'] = covid_melt_df.groupby(['countyFIPS'])['value'].diff().fillna(0)
# merge the covid data with the population data
usafacts_data = pd.merge(
    left=covid_melt_df, 
    right=pop_df[['countyFIPS','State','population']],
    how="left",
    on=['countyFIPS','State']
)
# remove 'statewide unallocated' from County Name
usafacts_data = usafacts_data.drop(usafacts_data[usafacts_data['County Name'] == 'Statewide Unallocated'].index)
# drop StateFIPS, value
usafacts_data = usafacts_data.drop(['countyFIPS','StateFIPS','value'],axis=1)
# rename variable to 'covid_date'
usafacts_data = usafacts_data.rename(columns={'variable':'covid_date'})
# create a county variable without the word 'county'
usafacts_data['County'] = usafacts_data['County Name'].str.split(' ',expand=True)[0]

# load crash data into dataframe
crashes_df = pd.read_csv('/home/gopherguy14/datasets/US_Accidents_Dec20.csv')
# convert start_time to datetime and save date component into new column
crashes_df['Start_Date'] = crashes_df['Start_Time'].astype("datetime64[ns]").dt.date
# drop old county name column
usafacts_data = usafacts_data.drop(columns='County Name')